In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:3


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [6]:
u0 = 50.0
loss_thresh = 0.1
label = "inv_HT_swish"

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [7]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
    
max_reps = 10 #10
max_iter = 200 #75

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []

lambda1_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):
    print(label)
    'Generate Training data'
    print(reps)
    torch.manual_seed(reps*36)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []   
    beta_val = []

    lambda1_val = []

    N_f = 50000 #Total number of collocation points 
    N_train = 5000

    layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    beta_full.append(beta_val)

    lambda1_full.append(lambda1_val)


    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full,"lambda1": lambda1_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic) 


inv_HT_swish
0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 854.39606 Test MSE 857.706490676958 Test RE 0.4930131678103855 Lambda1 0.09474899
1 Train Loss 815.4092 Test MSE 815.2977084481136 Test RE 0.48067029225326185 Lambda1 0.41177243
2 Train Loss 569.86536 Test MS

77 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
78 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
79 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
80 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
81 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
82 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
83 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
84 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
85 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
86 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
87 Train Loss 29.660

160 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
161 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
162 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
163 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
164 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
165 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
166 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
167 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
168 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
169 Train Loss 29.660843 Test MSE 33.31379060260357 Test RE 0.09716306283147874 Lambda1 0.93747133
170 Train 

36 Train Loss 29.928612 Test MSE 33.422668215921554 Test RE 0.09732170963652381 Lambda1 0.9389247
37 Train Loss 29.908829 Test MSE 33.39254459431444 Test RE 0.09727784205523946 Lambda1 0.9455327
38 Train Loss 29.892931 Test MSE 33.42682478351306 Test RE 0.09732776109365698 Lambda1 0.95105153
39 Train Loss 29.871616 Test MSE 33.3928826603178 Test RE 0.0972783344742576 Lambda1 0.96341825
40 Train Loss 29.840893 Test MSE 33.406255271157676 Test RE 0.09729781070684738 Lambda1 0.9625426
41 Train Loss 29.812746 Test MSE 33.415504500739075 Test RE 0.09731127925486564 Lambda1 0.944656
42 Train Loss 29.799517 Test MSE 33.381118138089604 Test RE 0.09726119708070892 Lambda1 0.94193804
43 Train Loss 29.784088 Test MSE 33.383532906796525 Test RE 0.09726471492344306 Lambda1 0.9408504
44 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
45 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
46 Train Loss 29.779665 T

121 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
122 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
123 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
124 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
125 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
126 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
127 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
128 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
129 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
130 Train Loss 29.779665 Test MSE 33.39262363422588 Test RE 0.09727795718314548 Lambda1 0.9297644
131 Train Loss 29.77

...

20 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
21 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
22 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
23 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
24 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
25 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
26 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
27 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
28 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
29 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
30 Train Loss 854.52

104 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
105 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
106 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
107 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
108 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
109 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
110 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
111 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
112 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
113 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
114 Train 

187 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
188 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
189 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
190 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
191 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
192 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
193 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
194 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
195 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
196 Train Loss 854.525 Test MSE 857.6885200748483 Test RE 0.4930080029958612 Lambda1 -0.0013049756
197 Train 

62 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
63 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
64 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
65 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
66 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
67 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
68 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
69 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
70 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
71 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
72 Train L

145 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
146 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
147 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
148 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
149 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
150 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
151 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
152 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
153 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688
154 Train Loss 854.52704 Test MSE 857.6824494782511 Test RE 0.4930062582727298 Lambda1 0.0031461688


In [13]:
import scipy.io as sio

In [14]:
for tune_reps in range(5):
    label = "inv_HT_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: 'inv_HT_swish_tune0.mat'